In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Check

In [ ]:
data = pd.read_csv("/kaggle/input/biomechanical-features-of-orthopedic-patients/column_3C_weka.csv")

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.info

In [ ]:
data.describe()

In [ ]:
data.isnull().any()

In [ ]:
data['class'].unique()

In [ ]:
data['class']  = [0 if each =="Normal" else 1 for each in data['class']]

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data['class'].unique()

In [ ]:
y = data['class'].values
x_data = data.loc[:,data.columns != 'class']

In [ ]:
x = (x_data - np.min(x_data))/(np.max(x_data)- np.min(x_data))

In [ ]:
x

# Data Train and Test Split

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train ,y_test = train_test_split(x,y ,test_size = 0.2 ,random_state=42 )

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
x_train = x_train.T
x_train.shape

In [ ]:
x_test = x_test.T
x_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

# LR

In [ ]:
def initialize_weights_and_bias(dimension):
    w = np.full((dimension,1),0.01)
    b = 0.0
    return w,b

In [ ]:
def sigmoid(z):
    y_head = 1/(1+ np.exp(-z)) 
    return y_head

In [ ]:
def forward_backward_propagation(w,b,x_train,y_train):
    z = np.dot(w.t,x_train)+b
    y_head=sigmoid(z)
    loss=-y_train*np.log(y_head-(1-y_train)*np.log(1-y_head))
    cost=(np.sum(loss)/x_train.shape[1])
    
    deriative_weight = np.dot(x_train,((y_head-y_train).T))/x_train.shape[1]
    deriative_bias = np.sum(y_head-y_train)/x_train.shape[1]
    gradients={"deriative_weight": deriative_weight, "deriative_bias": deriative_bias}
    
    return cost,gradient

In [ ]:
def update(w,b,x_train,y_train,learning_rate, number_of_iteration):
    cost_list=[]
    cost_list2=[]
    index=[]
    
    for i in range(number_of_iteration):
        cost,gradient= forward_backward_propagation(w,b,x_train,y_train)
        cost_list.append(cost)
        w = w- learning_rate*gradient["deriative_weight"]
        b = b- learning_rate*gradient["deriative_bias"]
        if i %5 == 0:
            cost_list2.append(cost)
            index.append(i)
            print("cost after iteration %i: %f" %(i,cost))
    
    parameters = {"weight": w, "bias": b}
    plt.plot(index,cost_list2)
    plt.xticks(index, rotation='vertical')
    plt.xlabel("Number of iteration")
    plt.ylabel("Cost")
    plt.show()
    
    return parameters,gradients,cost_list

In [ ]:
def predict(w,b,x_test):
    z = sigmoid(np.dot(w.T,x_test)+b)
    y_prediction= np.zeros((1,x_test.shape[1]))
    
    for i in range(z.shape[1]):
        if z[0,i] <=0.5:
            y_prediction[0,i]=0
        else :
            y_prediction[0,i]=1
    
    return y_prediction

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train.T,y_train.T)


In [ ]:
print("test accuracy {}".format(lr.score(x_test.T,y_test.T)))

# K-Nearest Neighbour(KNN) Classification

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(x_train.T, y_train.T)

prediction= knn.predict(x_test.T)
print("{} nn score:{} ".format(15,knn.score(x_test.T,y_test.T)))

In [ ]:
score_list=[]
for each in range (1,20):
    knn2= KNeighborsClassifier(n_neighbors=each)
    knn2.fit(x_train.T,y_train.T)
    score_list.append(knn2.score(x_test.T,y_test.T))

plt.plot(range(1,20),score_list)
plt.xlabel("k values")
plt.ylabel("acuuracy")
plt.show()